In [72]:
import cv2
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import numpy as np
import json
import random
import os
import re
import unicodedata


In [88]:
# Load JSON


def generate_labels(
    n_key_map={
        "date": 3_000,
        "designation": 65_000,
        "variety": 2_000,
        "region": 2_000,
        "province": 3_000,
        "country": 2_000,
        "winery": 8_000
    }       
):
    wine_data = json.load(open("./assets/vin_db.json", encoding="utf-8"))
    key_list = ["title", "designation", "variety", "region_1", "province", "country", "winery"]
    wine_data = [{k:d[k] for k in key_list} for d in wine_data]
    for wine in wine_data: 
        wine["title"] = re.sub("[^0-9]", "", wine["title"]) # Remove non alpha numeric characters, we only want the dates
        wine["date"] = wine.pop("title") # Rename key
        wine["region"] = wine.pop("region_1") # Rename key
    labels = {}
    for key in wine_data[0].keys():
        labels[key] = list(filter(None, [wine[key] for wine in wine_data] )) # Remove all None values
        labels[key] = list(map(lambda x: unicodedata.normalize("NFKD", re.sub(r'[^\x00-\x7F]', '', x).replace("\r\n", '').replace('\xad', '')), labels[key])) # Remove all \r and \n
        random.shuffle(labels[key]) # Shuffle
    for i in range(len(labels["designation"])): # split by 3 spaces and add remaning data at the end
        splitted_designtation = labels["designation"][i].split(' ')
        part1 = ' '.join(splitted_designtation[:3])
        part2 = ' '.join(splitted_designtation[3:])
        labels["designation"][i] = part1
        if type(part2) == str and len(part2) > 0:
            labels["designation"].append(part2)
    random.shuffle(labels["designation"]) # Shuffle again
    res_labels = []
    for key in n_key_map.keys(): # Cut to wanted length
        res_labels = res_labels + labels[key][:n_key_map[key]]
    return res_labels

def generate_images(labels, path):
    return


# for i in range(len(labels)):
#     print(f"{i+1}----{labels[i]}")


# labels = []
# for data in wine_data:
#     for key in data.keys():
#         if type(data[key]) == str and len(data[key]) > 0:
#             labels.append(data[key])
# print(type(labels))
# characetrs = sorted(list(set(char for label in labels for char in label)))
# print(len(characetrs))
# print(characetrs)
# font_files = os.listdir("./assets/fonts/")


# # Load image
# image = Image.new(
#     'RGB',
#     (150, 25),
#     (random.randint(175, 255), random.randint(175, 255), random.randint(175, 255))
# )
# draw = ImageDraw.Draw(image)

# # Load font
# font_size = 20
# font = ImageFont.truetype("./assets/fonts/" + random.choice(font_files), font_size)
# print(len(wine_data))
# # Draw text on image
# text = random.choice(wine_data)["title"]
# draw.text((150, 25), str(text), font=font, fill=(random.randint(0, 0), random.randint(0, 0), random.randint(0, 0)))

# # Convert to image OpenCV
# image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2RGBA)

# # Save image on disk
# image = Image.fromarray(image)
# image.show()
# # image.save(f"./OUTPUT{i}.png")

[' ', '!', '"', '#', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
84
85000


In [3]:




    # cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

(0, 0, 3456, 4608)
(0, 0, 3456, 4060)


: 

In [ ]:




    # cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

(0, 0, 3456, 4608)
(0, 0, 3456, 4060)


: 